## Парсер hh.ru

In [1]:
# Импорт класса BeautifulSoup из библиотеки bs4
from bs4 import BeautifulSoup 
# Импорт модуля requests для отправки HTTP-запросов
import requests  
# Импорт библиотеки numpy для работы с массивами и числами
import numpy as np  
# Импорт библиотеки pandas для работы с данными в виде таблиц
import pandas as pd  
# Импорт модуля re для работы с регулярными выражениями
import re  
# Импорт модуля warnings для управления предупреждениями
import warnings  

# Отключение предупреждений
warnings.filterwarnings('ignore')  


### Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайта HH. Приложение должно анализировать все страницы сайта.

**Получившийся список должен содержать в себе минимум:**
1. Наименование вакансии.
2. Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
3. Ссылку на саму вакансию.
4. Сайт, откуда собрана вакансия.
5. По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение).
6. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [2]:
# Определяем переменную для хранения данных
PREPARED_FILE_PATH = 'hh_parsed.csv'

# Заголовки для HTTP-запросов, чтобы представлять себя как обычный веб-браузер
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) \
    Chrome/114.0.0.0 Safari/537.36'
}

# Ввод интересующей должности
desired_job = input('Введите интересующую должность: ')

# Параметры запроса
params = {
    "text": desired_job,
    'enable_snippets': 'false',
    'per_page': 100,
    'page': 0, # Стартовая страница
    'area': 1, # 'false' если поиск необходимо осуществить по всей Россиии (Москва - 1)
    'search_period': 7 # Период, за который выводятся значения
}

# URL для запроса
url = "https://hh.ru"

# Строка поиска
search_line = '/search/vacancy?text=&'

# Создание сессии для отправки запросов
session = requests.Session()

# Список для хранения информации о вакансиях
job_list = []

# Словарь для хранения информации о зарплатах
salary_dict = {}

while True:
    # Отправка запроса на страницу поиска
    response = requests.get(url + search_line, headers=headers, params=params)

    # Создание объекта BeautifulSoup для парсинга HTML-кода
    soup = BeautifulSoup(response.text, "html.parser")

    # Поиск всех элементов с классом 'serp-item' (вакансии)
    jobs = soup.find_all('div', {'class': 'serp-item'})

    # Если вакансий нет, прерываем цикл
    if not jobs:
        break

    # Обход каждой вакансии
    for job in jobs:
        # Создание словаря для хранения информации о вакансии
        jobs_info = {}

        # Извлечение названия и ссылки на вакансию
        info = job.find('a', {'class': 'serp-item__title'})
        name = info.text
        link = info.get('href')

        # Извлечение идентификатора вакансии
        start_index = link.find("vacancy/") + len("vacancy/")
        end_index = link.find("?")
        vacancy_id = link[start_index:end_index]

        # Извлечение местоположения вакансии
        location = job.find('div', {'data-qa': "vacancy-serp__vacancy-address"}).text
        
        # Извлечение требуемого опыта работы
        experience = job.find('div', {'data-qa': 'vacancy-serp__vacancy-work-experience'}).text
        
        # Извлечение компании вакансии
        employer_info = job.find('a', {'data-qa': "vacancy-serp__vacancy-employer"})
        employer = employer_info.text
        
        # Излечение ссылки на страницу компании
        employer_page = employer_info.get('href')
        employer_page = url + employer_page

        # Извлечение информации о зарплате и сохранение в словаре
        salary_dict[vacancy_id] = job.find_all(string=True)

        # Добавление информации о вакансии в список
        source_site = url
        jobs_info['name'] = name
        jobs_info['link'] = link
        jobs_info['source_site'] = source_site
        jobs_info['vacancy_id'] = vacancy_id
        jobs_info['location'] = location
        jobs_info['employer'] = employer
        jobs_info['employer_page'] = employer_page
        jobs_info['experience'] = experience
        job_list.append(jobs_info)

    # Вывод номера обработанной страницы
    print(f"Обработана страница №{params['page']}")

    # Увеличение значения параметра 'page' для перехода на следующую страницу
    params['page'] += 1

print('Этот этап завершен')


Введите интересующую должность: аналитик
Обработана страница №0
Обработана страница №1
Обработана страница №2
Обработана страница №3
Обработана страница №4
Обработана страница №5
Обработана страница №6
Обработана страница №7
Обработана страница №8
Обработана страница №9
Обработана страница №10
Обработана страница №11
Обработана страница №12
Обработана страница №13
Обработана страница №14
Обработана страница №15
Обработана страница №16
Обработана страница №17
Обработана страница №18
Обработана страница №19
Обработана страница №20
Обработана страница №21
Обработана страница №22
Обработана страница №23
Обработана страница №24
Обработана страница №25
Обработана страница №26
Обработана страница №27
Обработана страница №28
Обработана страница №29
Обработана страница №30
Обработана страница №31
Обработана страница №32
Обработана страница №33
Обработана страница №34
Обработана страница №35
Обработана страница №36
Обработана страница №37
Обработана страница №38
Обработана страница №39
Этот этап

In [3]:
# Список для сохранения извлеченных зарплат
salaries_extracted = []

# Список для сохранения промежуточных значений
holder_list = []

# Список для сохранения валют
currency = []

# Шаблон для поиска зарплатных значений
pattern = r'[^\s]+\u202f[^\s]+'

for key, value in salary_dict.items():
    # Удаление строки с информацией об опыте работы, если она присутствует
    if any('Опыт' in l for l in value):
        value.remove(next(l for l in value if 'Опыт' in l))
    
    # Проверка наличия символа '\u202f' в строке значений
    if any('\u202f' in s for s in value):
        # Итерация по каждой строке значений
        for string in value:
            # Поиск зарплатных значений в строке с помощью регулярного выражения
            matches = re.findall(pattern, string)

            if matches:
                # Обработка найденных зарплатных значений
                for match in matches:
                    # Удаление символа '\u202f' и добавление в список holder_list
                    num = match.replace('\u202f', '')
                    holder_list.append(num)
                
                # Проверка количества извлеченных зарплатных значений
                if len(holder_list) == 1:
                    # Проверка наличия ключевых слов 'до' и 'от' в строке значений
                    if any('до ' in s for s in value): 
                        # Добавление словаря с минимальной и максимальной зарплатой в список salaries_extracted
                        salaries_extracted.append({'vacancy_id': key, 'min_salary': np.nan, 'max_salary': num})
                    elif any('от ' in s for s in value):
                        salaries_extracted.append({'vacancy_id': key, 'min_salary': num, 'max_salary': np.nan})
                elif any('от ' in s for s in value) and any(' до ' in s for s in value):
                    salaries_extracted.append({'vacancy_id': key, 'min_salary': holder_list[0], \
                                               'max_salary': holder_list[1]})
                else:
                    salaries_extracted.append({'vacancy_id': key, 'min_salary': holder_list[0], \
                                               'max_salary': holder_list[1]})
                
                # Сброс списка holder_list
                holder_list = []
        
    else:
        # Добавление словаря с информацией о зарплате (с отсутствующими значениями) в список salaries_extracted
        salaries_extracted.append({'vacancy_id': key, 'min_salary': np.nan, 'max_salary': np.nan})
        
        
for key, value in salary_dict.items():
    # Проверка наличия символа '₽' в строке значений
    if any('₽' in s for s in value):
        # Добавление словаря с валютой 'RUB' в список currency
        currency.append({'vacancy_id': key, 'currency': 'RUB'})
    # Проверка наличия символа '$' в строке значений
    elif any('$' in s for s in value):
        # Добавление словаря с валютой 'USD' в список currency
        currency.append({'vacancy_id': key, 'currency': 'USD'})
    # Проверка наличия символа '€' в строке значений
    elif any('€' in s for s in value):
        # Добавление словаря с валютой 'EUR' в список currency
        currency.append({'vacancy_id': key, 'currency': 'EUR'})
    else:
        # Добавление словаря с отсутствующей валютой в список currency
        currency.append({'vacancy_id': key, 'currency': np.nan})



# Создание DataFrame для списка с информацией о должностях
jobs_df = pd.DataFrame(job_list)

# Создание DataFrame для списка с извлеченными зарплатами
salaries_df = pd.DataFrame(salaries_extracted)

# Создание DataFrame для списка с валютами
currency_df = pd.DataFrame(currency)

# Объединение DataFrame по столбцу 'vacancy_id' для получения итогового DataFrame
final_df = jobs_df.merge(salaries_df, on='vacancy_id').merge(currency_df, on='vacancy_id')

# Сохранение DataFrame в csv файл
final_df.to_csv(PREPARED_FILE_PATH)


In [4]:
final_df

,name,link,source_site,vacancy_id,location,employer,employer_page,experience,min_salary,max_salary,currency
0,Риск-аналитик,https://hh.ru/vacancy/83454840?from=vacancy_se...,https://hh.ru,83454840,Москва,ЦЕНТРОФИНАНС,https://hh.ru/employer/1103383?hhtmFrom=vacanc...,Опыт от 1 года до 3 лет,110000,150000,RUB
1,Системный аналитик,https://hh.ru/vacancy/83455870?from=vacancy_se...,https://hh.ru,83455870,Москва,АО Рут Код,https://hh.ru/employer/8642172?hhtmFrom=vacanc...,Опыт от 3 до 6 лет,NaN,NaN,NaN
2,Маркетолог- аналитик по работе с маркетплейсами,https://hh.ru/vacancy/83343120?from=vacancy_se...,https://hh.ru,83343120,"Москва, Крылатское",СИМБАТ,https://hh.ru/employer/7788?hhtmFrom=vacancy_s...,Опыт от 1 года до 3 лет,120000,NaN,RUB
3,Младший аналитик данных,https://hh.ru/vacancy/83114951?from=vacancy_se...,https://hh.ru,83114951,"Москва, Белорусская и еще 2",ООО Верме,https://hh.ru/employer/3629847?hhtmFrom=vacanc...,Без опыта,NaN,NaN,NaN
4,Analyst / Аналитик,https://hh.ru/vacancy/82985141?from=vacancy_se...,https://hh.ru,82985141,Москва,ООО DataGo!,https://hh.ru/employer/9035637?hhtmFrom=vacanc...,Опыт от 3 до 6 лет,NaN,250000,RUB
...,...,...,...,...,...,...,...,...,...,...,...
795,Финансовый менеджер / контролер (удаленно),https://hh.ru/vacancy/83020519?from=vacancy_se...,https://hh.ru,83020519,Москва,SVMOSCOW,https://hh.ru/employer/622709?hhtmFrom=vacancy...,Опыт от 3 до 6 лет,60000,60000,RUB
796,Продакт менеджер / Менеджер по продукту / Prod...,https://hh.ru/vacancy/82997064?from=vacancy_se...,https://hh.ru,82997064,Москва,Funtastique,https://hh.ru/employer/4500326?hhtmFrom=vacanc...,Опыт от 1 года до 3 лет,120000,NaN,RUB
797,Помощник маркетолога,https://hh.ru/vacancy/82749759?from=vacancy_se...,https://hh.ru,82749759,Москва,ООО Медиа ГУРУ,https://hh.ru/employer/1020357?hhtmFrom=vacanc...,Опыт от 1 года до 3 лет,40000,40000,RUB
798,Системный аналитик (Федеральный проект),https://hh.ru/vacancy/81902995?from=vacancy_se...,https://hh.ru,81902995,Москва,CUSTIS,https://hh.ru/employer/2537?hhtmFrom=vacancy_s...,Опыт от 3 до 6 лет,NaN,NaN,NaN
